# Neo4j Lab 11: Python Driver & Service Architecture
## Part 4: Testing Framework & Integration Tests

**Duration:** 10 minutes  
**Objective:** Implement comprehensive testing strategies including unit tests, integration tests, and error handling validation

---

## Overview

This notebook covers:
- Unit testing with pytest
- Data validation testing
- Integration testing with live database
- Error handling validation
- Performance testing
- Data consistency verification

## Cell 1: Import Testing Dependencies

Import required testing frameworks and utilities.

In [ ]:
# Cell 1: Import testing dependencies
# Note: Make sure you've run notebooks 01-03 first

# If previous components are not available, uncomment and run:
# %run 01_python_driver_setup_and_basics.ipynb
# %run 02_pydantic_models_and_validation.ipynb
# %run 03_repository_and_service_layer.ipynb

import pytest
from unittest.mock import Mock, patch
import asyncio
from typing import Any, Dict
import time

print("🧪 IMPLEMENTING TESTING FRAMEWORK:")
print("=" * 50)

## Cell 2: Unit Tests - Data Validation

Test Pydantic model validation rules to ensure data integrity.

In [ ]:
# Cell 2: Unit tests for data validation

class TestDataValidation:
    """Test suite for Pydantic model validation"""
    
    def test_customer_creation_validation(self):
        """Test customer data validation"""
        print("\nTest 1: Customer Creation Validation")
        print("-" * 50)
        
        try:
            # Test 1.1: Valid customer
            valid_customer = CustomerCreate(
                customer_id="CUST-TEST-001",
                first_name="John",
                last_name="Doe",
                email="john.doe@test.com",
                phone="+1234567890",
                date_of_birth=date(1990, 1, 1)
            )
            assert valid_customer.customer_id == "CUST-TEST-001"
            print("✓ Test 1.1: Valid customer creation passed")
            
            # Test 1.2: Invalid customer ID format
            try:
                invalid_customer = CustomerCreate(
                    customer_id="INVALID-001",
                    first_name="John",
                    last_name="Doe",
                    email="john.doe@test.com",
                    date_of_birth=date(1990, 1, 1)
                )
                print("✗ Test 1.2: Invalid customer ID test failed - should have raised error")
            except ValueError:
                print("✓ Test 1.2: Invalid customer ID validation passed")
            
            # Test 1.3: Invalid age (too young)
            try:
                young_customer = CustomerCreate(
                    customer_id="CUST-TEST-002",
                    first_name="Too",
                    last_name="Young",
                    email="young@test.com",
                    date_of_birth=date(2010, 1, 1)
                )
                print("✗ Test 1.3: Age validation test failed - should have raised error")
            except ValueError:
                print("✓ Test 1.3: Age validation (too young) passed")
            
            # Test 1.4: Invalid age (too old)
            try:
                old_customer = CustomerCreate(
                    customer_id="CUST-TEST-003",
                    first_name="Too",
                    last_name="Old",
                    email="old@test.com",
                    date_of_birth=date(1900, 1, 1)
                )
                print("✗ Test 1.4: Age validation test failed - should have raised error")
            except ValueError:
                print("✓ Test 1.4: Age validation (too old) passed")
            
            # Test 1.5: Invalid email
            try:
                invalid_email = CustomerCreate(
                    customer_id="CUST-TEST-004",
                    first_name="Bad",
                    last_name="Email",
                    email="not-an-email",
                    date_of_birth=date(1990, 1, 1)
                )
                print("✗ Test 1.5: Email validation test failed - should have raised error")
            except ValueError:
                print("✓ Test 1.5: Email validation passed")
            
            print("\n✓ All customer validation tests passed!")
            return True
                
        except Exception as e:
            print(f"\n✗ Customer validation tests failed: {e}")
            return False
    
    def test_policy_validation(self):
        """Test policy data validation"""
        print("\nTest 2: Policy Validation")
        print("-" * 50)
        
        try:
            # Test 2.1: Valid policy
            valid_policy = PolicyCreate(
                policy_number="POL-AUTO-001",
                policy_type=PolicyType.AUTO,
                customer_id="CUST-123456",
                effective_date=date(2025, 1, 1),
                expiration_date=date(2026, 1, 1),
                premium_amount=150.00,
                coverage_amount=25000.00,
                deductible=500.00
            )
            assert valid_policy.policy_type == PolicyType.AUTO
            print("✓ Test 2.1: Valid policy creation passed")
            
            # Test 2.2: Invalid policy number
            try:
                invalid_policy = PolicyCreate(
                    policy_number="INVALID",
                    policy_type=PolicyType.AUTO,
                    customer_id="CUST-123456",
                    effective_date=date(2025, 1, 1),
                    expiration_date=date(2026, 1, 1),
                    premium_amount=150.00,
                    coverage_amount=25000.00
                )
                print("✗ Test 2.2: Policy number validation failed - should have raised error")
            except ValueError:
                print("✓ Test 2.2: Policy number validation passed")
            
            # Test 2.3: Invalid date range
            try:
                invalid_dates = PolicyCreate(
                    policy_number="POL-AUTO-002",
                    policy_type=PolicyType.AUTO,
                    customer_id="CUST-123456",
                    effective_date=date(2025, 12, 31),
                    expiration_date=date(2025, 1, 1),  # Before effective
                    premium_amount=150.00,
                    coverage_amount=25000.00
                )
                print("✗ Test 2.3: Date validation failed - should have raised error")
            except ValueError:
                print("✓ Test 2.3: Policy date validation passed")
            
            # Test 2.4: Invalid premium amount
            try:
                invalid_premium = PolicyCreate(
                    policy_number="POL-AUTO-003",
                    policy_type=PolicyType.AUTO,
                    customer_id="CUST-123456",
                    effective_date=date(2025, 1, 1),
                    expiration_date=date(2026, 1, 1),
                    premium_amount=-100.00,  # Negative
                    coverage_amount=25000.00
                )
                print("✗ Test 2.4: Premium validation failed - should have raised error")
            except ValueError:
                print("✓ Test 2.4: Premium amount validation passed")
            
            print("\n✓ All policy validation tests passed!")
            return True
                
        except Exception as e:
            print(f"\n✗ Policy validation tests failed: {e}")
            return False
    
    def test_claim_validation(self):
        """Test claim data validation"""
        print("\nTest 3: Claim Validation")
        print("-" * 50)
        
        try:
            # Test 3.1: Valid claim
            valid_claim = ClaimCreate(
                claim_number="CLM-2025-001",
                policy_number="POL-AUTO-001",
                claim_date=date.today(),
                incident_date=date.today() - timedelta(days=1),
                claim_amount=1000.00,
                description="Test claim with valid description that is long enough"
            )
            print("✓ Test 3.1: Valid claim creation passed")
            
            # Test 3.2: Invalid claim number
            try:
                invalid_claim = ClaimCreate(
                    claim_number="INVALID",
                    policy_number="POL-AUTO-001",
                    claim_date=date.today(),
                    incident_date=date.today(),
                    claim_amount=1000.00,
                    description="Test claim description"
                )
                print("✗ Test 3.2: Claim number validation failed - should have raised error")
            except ValueError:
                print("✓ Test 3.2: Claim number validation passed")
            
            # Test 3.3: Invalid incident date (after claim date)
            try:
                invalid_dates = ClaimCreate(
                    claim_number="CLM-2025-002",
                    policy_number="POL-AUTO-001",
                    claim_date=date.today(),
                    incident_date=date.today() + timedelta(days=1),  # Future
                    claim_amount=1000.00,
                    description="Test claim description"
                )
                print("✗ Test 3.3: Incident date validation failed - should have raised error")
            except ValueError:
                print("✓ Test 3.3: Incident date validation passed")
            
            # Test 3.4: Invalid description (too short)
            try:
                short_desc = ClaimCreate(
                    claim_number="CLM-2025-003",
                    policy_number="POL-AUTO-001",
                    claim_date=date.today(),
                    incident_date=date.today(),
                    claim_amount=1000.00,
                    description="Short"  # Too short
                )
                print("✗ Test 3.4: Description validation failed - should have raised error")
            except ValueError:
                print("✓ Test 3.4: Description length validation passed")
            
            print("\n✓ All claim validation tests passed!")
            return True
                
        except Exception as e:
            print(f"\n✗ Claim validation tests failed: {e}")
            return False

# Run validation tests
print("\n🧪 RUNNING VALIDATION TESTS:")
print("=" * 50)

test_validation = TestDataValidation()
results = [
    test_validation.test_customer_creation_validation(),
    test_validation.test_policy_validation(),
    test_validation.test_claim_validation()
]

print("\n" + "=" * 50)
print(f"VALIDATION TEST RESULTS: {sum(results)}/{len(results)} passed")
if all(results):
    print("🎉 All validation tests passed!")
else:
    print("⚠ Some validation tests failed")
print("=" * 50)

## Cell 3: Connection Resilience Tests

Test database connection handling and error recovery.

In [ ]:
# Cell 3: Connection resilience and error handling tests

print("\n🧪 TESTING CONNECTION RESILIENCE:")
print("=" * 50)

def test_database_connection_resilience():
    """Test connection resilience and retry logic"""
    try:
        print("\nTest 4: Database Connection Resilience")
        print("-" * 50)
        
        # Test 4.1: Connection health check
        health_status = connection_manager.health_check()
        assert health_status['status'] in ['healthy', 'unhealthy']
        print(f"✓ Test 4.1: Health check returned status: {health_status['status']}")
        
        # Test 4.2: Parameterized query execution
        result = connection_manager.execute_query(
            "RETURN $test_param as result",
            {"test_param": "test_value"}
        )
        assert len(result) == 1
        assert result[0]['result'] == 'test_value'
        print("✓ Test 4.2: Parameterized query execution passed")
        
        # Test 4.3: Query with multiple results
        result = connection_manager.execute_query("""
            UNWIND range(1, 5) as num
            RETURN num
        """)
        assert len(result) == 5
        print("✓ Test 4.3: Multiple result query passed")
        
        # Test 4.4: Connection metrics
        assert connection_manager._successful_queries > 0
        print(f"✓ Test 4.4: Connection metrics tracked ({connection_manager._successful_queries} successful queries)")
        
        print("\n✓ All connection resilience tests passed!")
        return True
        
    except Exception as e:
        print(f"\n✗ Connection resilience tests failed: {e}")
        import traceback
        traceback.print_exc()
        return False

# Run connection tests
connection_result = test_database_connection_resilience()

print("\n" + "=" * 50)
print(f"CONNECTION TEST RESULT: {'PASSED' if connection_result else 'FAILED'}")
print("=" * 50)

## Cell 4: Integration Tests

Test complete workflows with live database operations.

In [ ]:
# Cell 4: Integration tests with live database

print("\n📈 PERFORMING INTEGRATION TESTING:")
print("=" * 50)

class IntegrationTestSuite:
    """Live integration tests with actual Neo4j database"""
    
    def __init__(self, service: InsuranceService):
        self.service = service
        self.test_data = []
        self.logger = logging.getLogger(self.__class__.__name__)
    
    def run_full_integration_test(self) -> Dict[str, Any]:
        """Run comprehensive integration test"""
        test_results = {
            "customer_creation": False,
            "policy_creation": False,
            "claim_processing": False,
            "customer_360_view": False,
            "data_consistency": False,
            "performance_metrics": {}
        }
        
        try:
            # Test 1: Customer and Policy Creation
            print("\nTest 5: Customer and Policy Creation (Integration)")
            print("-" * 50)
            start_time = time.time()
            customer_result = self._test_customer_creation()
            test_results["customer_creation"] = customer_result is not None
            test_results["performance_metrics"]["customer_creation_ms"] = round((time.time() - start_time) * 1000, 2)
            
            if customer_result:
                customer_id = customer_result["customer"]["customerId"]
                policy_number = customer_result["policy"]["policyNumber"]
                test_results["policy_creation"] = True
                
                # Test 2: Claim Processing
                print("\nTest 6: Claim Processing (Integration)")
                print("-" * 50)
                start_time = time.time()
                claim_result = self._test_claim_processing(policy_number)
                test_results["claim_processing"] = claim_result is not None
                test_results["performance_metrics"]["claim_processing_ms"] = round((time.time() - start_time) * 1000, 2)
                
                # Test 3: Customer 360 View
                print("\nTest 7: Customer 360 View (Integration)")
                print("-" * 50)
                start_time = time.time()
                view_result = self._test_customer_360_view(customer_id)
                test_results["customer_360_view"] = view_result is not None
                test_results["performance_metrics"]["customer_360_view_ms"] = round((time.time() - start_time) * 1000, 2)
                
                # Test 4: Data Consistency
                print("\nTest 8: Data Consistency (Integration)")
                print("-" * 50)
                consistency_result = self._test_data_consistency(customer_id)
                test_results["data_consistency"] = consistency_result
            
            return test_results
            
        except Exception as e:
            self.logger.error(f"Integration test failed: {e}")
            return test_results
        
        finally:
            # Cleanup test data
            self._cleanup_test_data()
    
    def _test_customer_creation(self) -> Optional[Dict[str, Any]]:
        """Test customer and policy creation"""
        try:
            timestamp = int(time.time() * 1000)  # Use milliseconds for uniqueness
            
            customer_data = CustomerCreate(
                customer_id=f"CUST-INTEG-{timestamp}",
                first_name="Integration",
                last_name="Test",
                email=f"integration.test.{timestamp}@example.com",
                phone="+1555123456",
                date_of_birth=date(1985, 5, 15),
                initial_contact_method="Integration Test",
                referral_source="Automated Testing"
            )
            
            policy_data = PolicyCreate(
                policy_number=f"POL-INTEG-{timestamp}",
                policy_type=PolicyType.AUTO,
                customer_id=customer_data.customer_id,
                effective_date=date.today(),
                expiration_date=date.today() + timedelta(days=365),
                premium_amount=125.50,
                coverage_amount=30000.00,
                deductible=750.00
            )
            
            result = self.service.create_customer_with_policy(customer_data, policy_data)
            self.test_data.append(customer_data.customer_id)
            
            print(f"✓ Customer and policy created: {customer_data.customer_id}")
            print(f"  Customer: {customer_data.first_name} {customer_data.last_name}")
            print(f"  Policy: {policy_data.policy_number} ({policy_data.policy_type})")
            return result
            
        except Exception as e:
            print(f"✗ Customer creation test failed: {e}")
            return None
    
    def _test_claim_processing(self, policy_number: str) -> Optional[Dict[str, Any]]:
        """Test claim processing functionality"""
        try:
            timestamp = int(time.time() * 1000)
            
            claim_data = ClaimCreate(
                claim_number=f"CLM-INTEG-{timestamp}",
                policy_number=policy_number,
                claim_date=date.today(),
                incident_date=date.today() - timedelta(days=1),
                claim_amount=2500.00,
                description="Integration test claim for minor vehicle damage during automated testing scenario"
            )
            
            result = self.service.process_claim(claim_data)
            
            print(f"✓ Claim processed: {claim_data.claim_number}")
            print(f"  Amount: ${claim_data.claim_amount:,.2f}")
            print(f"  Potential payout: ${result['business_analysis']['potential_payout']:,.2f}")
            return result
            
        except Exception as e:
            print(f"✗ Claim processing test failed: {e}")
            return None
    
    def _test_customer_360_view(self, customer_id: str) -> Optional[Dict[str, Any]]:
        """Test customer 360-degree view"""
        try:
            result = self.service.get_customer_360_view(customer_id)
            
            if "error" not in result:
                summary = result['summary']
                print(f"✓ Customer 360 view retrieved")
                print(f"  Policies: {summary['total_policies']}")
                print(f"  Claims: {summary['total_claims']}")
                print(f"  Risk score: {summary['latest_risk_score']:.1f}")
                return result
            else:
                print(f"✗ Customer 360 view failed: {result['error']}")
                return None
                
        except Exception as e:
            print(f"✗ Customer 360 view test failed: {e}")
            return None
    
    def _test_data_consistency(self, customer_id: str) -> bool:
        """Test data consistency across relationships"""
        try:
            consistency_query = """
            MATCH (c:Customer {customerId: $customer_id})
            OPTIONAL MATCH (c)-[:HOLDS]->(p:Policy)
            OPTIONAL MATCH (p)-[:COVERS]->(cl:Claim)
            
            RETURN c.customerId as customer_id,
                   count(DISTINCT p) as policy_count,
                   count(DISTINCT cl) as claim_count,
                   c.totalPolicies as customer_policy_count,
                   c.totalClaims as customer_claim_count
            """
            
            result = self.service.connection_manager.execute_query(
                consistency_query, 
                {"customer_id": customer_id}
            )
            
            if result:
                data = result[0]
                policy_consistent = data['policy_count'] == data['customer_policy_count']
                claim_consistent = data['claim_count'] == data['customer_claim_count']
                
                if policy_consistent and claim_consistent:
                    print("✓ Data consistency verified")
                    print(f"  Policies: {data['policy_count']} (consistent)")
                    print(f"  Claims: {data['claim_count']} (consistent)")
                    return True
                else:
                    print(f"✗ Data inconsistency detected")
                    print(f"  Policies: {data['policy_count']}/{data['customer_policy_count']}")
                    print(f"  Claims: {data['claim_count']}/{data['customer_claim_count']}")
                    return False
            else:
                print("✗ Customer not found for consistency check")
                return False
                
        except Exception as e:
            print(f"✗ Data consistency test failed: {e}")
            return False
    
    def _cleanup_test_data(self):
        """Clean up test data after testing"""
        try:
            for customer_id in self.test_data:
                cleanup_query = """
                MATCH (c:Customer {customerId: $customer_id})
                OPTIONAL MATCH (c)-[:HOLDS]->(p:Policy)
                OPTIONAL MATCH (p)-[:COVERS]->(cl:Claim)
                OPTIONAL MATCH (c)-[:HAS_RISK_ASSESSMENT]->(ra:RiskAssessment)
                DETACH DELETE c, p, cl, ra
                """
                
                self.service.connection_manager.execute_query(
                    cleanup_query, 
                    {"customer_id": customer_id}
                )
            
            if self.test_data:
                print(f"\n✓ Cleaned up {len(self.test_data)} test records")
            
        except Exception as e:
            print(f"\n⚠ Cleanup failed: {e}")

# Run integration tests
try:
    integration_tester = IntegrationTestSuite(insurance_service)
    test_results = integration_tester.run_full_integration_test()
    
    print("\n" + "="*50)
    print("INTEGRATION TEST RESULTS:")
    print("="*50)
    
    for test_name, passed in test_results.items():
        if test_name != "performance_metrics":
            status = "✓ PASS" if passed else "✗ FAIL"
            print(f"{test_name.replace('_', ' ').title()}: {status}")
    
    print("\nPERFORMANCE METRICS:")
    for metric, value in test_results["performance_metrics"].items():
        print(f"├─ {metric.replace('_', ' ').title()}: {value}")
    
    # Calculate overall success rate
    total_tests = len([t for t in test_results.keys() if t != "performance_metrics"])
    passed_tests = sum([1 for k, v in test_results.items() if k != "performance_metrics" and v])
    success_rate = (passed_tests / total_tests) * 100
    
    print(f"\nOVERALL SUCCESS RATE: {success_rate:.1f}% ({passed_tests}/{total_tests})")
    
    if success_rate >= 80:
        print("🎉 Integration testing completed successfully!")
    else:
        print("⚠ Some integration tests failed - review implementation")
    
except Exception as e:
    print(f"✗ Integration testing failed: {e}")
    import traceback
    traceback.print_exc()

print("=" * 50)

## Summary

In this notebook, you've:

1. ✅ Implemented unit tests for:
   - Customer data validation
   - Policy data validation
   - Claim data validation
   - Business rule enforcement

2. ✅ Tested connection resilience:
   - Health check functionality
   - Parameterized queries
   - Connection metrics tracking
   - Error recovery

3. ✅ Performed integration tests:
   - End-to-end customer creation
   - Policy and claim processing
   - Customer 360-degree view
   - Data consistency verification

4. ✅ Measured performance metrics:
   - Operation timing
   - Query execution time
   - Success/failure rates

5. ✅ Implemented proper test cleanup:
   - Automatic test data removal
   - Database state restoration

**Key Testing Principles:**
- Test early, test often
- Validate both positive and negative cases
- Measure performance
- Ensure data consistency
- Clean up test data

**Next Steps:** Proceed to `05_monitoring_and_production.ipynb` to implement production monitoring and health checks.